<a href="https://colab.research.google.com/github/GaurangRawat/NLP/blob/main/NLP_7%20Birdirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using Bidirectional LSTM
---

In [42]:
import pandas as pd

In [43]:
df = pd.read_csv('train.csv')

In [44]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [45]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [46]:
df.dropna()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [47]:
### get the independent features
X=df.drop('label', axis=1)

In [48]:
## get the dependent feature
y= df['label']

In [49]:
y.value_counts()

,count
label,
1,10413
0,10387


In [50]:
import tensorflow as tf

In [51]:
tf.__version__

'2.17.1'

In [52]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [53]:
## Vocabiulary Size
voc_size = 10000

In [54]:
messages = X.copy()

In [55]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [56]:
messages.reset_index(inplace=True)

In [57]:
import nltk
import re
from nltk.corpus import stopwords

In [58]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [73]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    title = messages['title'][i]

    # Check if the title is a valid string
    if not isinstance(title, str):
        title = str(title)  # Convert to string, or use `continue` to skip
    review = re.sub('[^a-zA-Z]', ' ', title)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [74]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

##One hot  Representation
---

In [76]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[9171, 8363, 5232, 2307, 9566, 6529, 4633, 13, 1155, 3159],
 [5222, 463, 7190, 9443, 9896, 8960, 6438],
 [2065, 9101, 2575, 9981],
 [8035, 4439, 1068, 629, 8188, 1348],
 [2407, 9896, 1409, 6564, 148, 3826, 9896, 3381, 7376, 8566],
 [5908,
  1533,
  6868,
  3777,
  3121,
  8418,
  4957,
  8526,
  6459,
  1436,
  3067,
  2215,
  7559,
  3900,
  6438],
 [7889, 7889, 3477, 6286, 4535, 3496, 6523, 3289, 1337, 418, 820, 9059],
 [4327, 2668, 3268, 7320, 1279, 8410, 3282, 7551, 9314, 9343, 6692],
 [7986, 4703, 2093, 5046, 8418, 7439, 487, 8377, 638, 4703, 9314, 9343, 6692],
 [5316, 2480, 2930, 2638, 6684, 5462, 8418, 2459, 9314, 9343, 6692],
 [911, 2857, 9506, 9267, 1356, 4437, 5158, 2968, 8418, 5173],
 [230, 5617, 606, 7096, 4599, 9719, 1008, 5355],
 [8934, 5119, 1023, 4367, 6709, 9087, 1242, 9976, 7949, 8280, 9982],
 [629, 2555, 9566, 4437, 8418, 6684],
 [1581, 971, 8301, 509, 2248, 9826, 1862, 2981, 1279],
 [8775, 2957, 2717, 3477, 7745, 553, 5661, 9314, 9343, 6692],
 [7209, 7518, 8554, 37

## Embedding Representation
---

In [77]:
sent_length = 20
embedded_docs=pad_sequences(onehot_repr, padding='post', maxlen=sent_length)
embedded_docs[0]

array([9171, 8363, 5232, 2307, 9566, 6529, 4633,   13, 1155, 3159,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [89]:
# Creating Model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.build(input_shape=(None, sent_length, ))
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 20, 40)              │         400,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 200)                 │         112,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             201 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 513,001 (1.96 MB)

 Trainable params: 513,001 (1.96 MB)

 Non-trainable params: 0 (0.00 B)

None


In [90]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [91]:
## train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [92]:
## Model Training
model.fit(X_train,y_train, validation_data=(X_test, y_test), epochs=10, batch_size = 32)

Epoch 1/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 24s 46ms/step - accuracy: 0.8397 - loss: 0.3322 - val_accuracy: 0.9295 - val_loss: 0.1709
Epoch 2/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 18s 40ms/step - accuracy: 0.9645 - loss: 0.0985 - val_accuracy: 0.9257 - val_loss: 0.1869
Epoch 3/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 21s 41ms/step - accuracy: 0.9817 - loss: 0.0623 - val_accuracy: 0.9288 - val_loss: 0.2073
Epoch 4/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 20s 41ms/step - accuracy: 0.9861 - loss: 0.0486 - val_accuracy: 0.9263 - val_loss: 0.2536
Epoch 5/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 20s 41ms/step - accuracy: 0.9892 - loss: 0.0365 - val_accuracy: 0.9267 - val_loss: 0.2423
Epoch 6/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 22s 44ms/step - accuracy: 0.9936 - loss: 0.0242 - val_accuracy: 0.9212 - val_loss: 0.3052
Epoch 7/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.9956 - loss: 0.0163 - val_accuracy: 0.9257 - val_loss: 0.3717
Epoch 8/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 21s 47ms/step - accuracy: 0.9966 - loss: 0.0107 - 

In [93]:
y_pred=model.predict(X_test)

215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step


In [94]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [95]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3136,  313],
       [ 223, 3192]])

In [96]:
print(accuracy_score(y_test,y_pred))

0.921911421911422
